In [29]:
import numpy as np
import pandas as pd

In [61]:
data = pd.read_csv("E:/baselines/ZZ500.csv")
code = "zz500"
# data = data.drop(columns=['Adj Close'])
# data = data.drop(columns=['prev_close', 'adjfactor'])
# data.rename(columns={'Date':'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume', 'company':'tic'}, inplace=True)
data.rename(columns={'dt':'date', 'kdcode':'tic'}, inplace=True)

In [62]:
data['date'] = pd.to_datetime(data['date'])
grouped_df = data.groupby('tic')
l = data['date'].nunique()
def filter_groups(group):
    return len(group) == l
df = grouped_df.filter(filter_groups)
df

,tic,date,close,open,high,low,turnover,volume
0,000009.SZ,2018-01-02,7.32,7.27,7.33,7.23,140497530.0,19269993.0
1,000009.SZ,2018-01-03,7.42,7.32,7.44,7.30,170636551.0,23092097.0
2,000009.SZ,2018-01-04,7.64,7.52,7.70,7.51,358089946.0,46992707.0
3,000009.SZ,2018-01-05,7.55,7.65,7.65,7.51,196259451.0,25990264.0
4,000009.SZ,2018-01-08,7.52,7.55,7.63,7.46,178511079.0,23703431.0
...,...,...,...,...,...,...,...,...
169007,603883.SH,2023-12-25,29.37,29.02,29.58,28.80,101007153.0,3457490.0
169008,603883.SH,2023-12-26,29.05,29.26,29.49,28.82,66158144.0,2272587.0
169009,603883.SH,2023-12-27,30.08,28.94,30.40,28.80,206474442.0,6939076.0
169010,603883.SH,2023-12-28,28.98,30.15,30.20,28.44,264520627.0,9053474.0


In [63]:
stock_num = df['tic'].nunique()
stock_num

116

In [64]:
stock_unique = df['tic'].unique()
'''
tic_sw_mapping = df[['tic', 'sw_kdcode_1']]
unique_mapping = tic_sw_mapping.groupby('tic').first().to_dict()['sw_kdcode_1']
unique_mapping
'''

"\ntic_sw_mapping = df[['tic', 'sw_kdcode_1']]\nunique_mapping = tic_sw_mapping.groupby('tic').first().to_dict()['sw_kdcode_1']\nunique_mapping\n"

In [65]:
df['date'].nunique()

1457

In [66]:
'''
test_idx的下标
'''
p = df['date'].unique()
np.where(p == "2023-01-03")[0][0]

1215

In [67]:
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# cols_to_normalize = ['close', 'open', 'high', 'low', 'volume']
cols_to_normalize = ['close', 'open', 'high', 'low', 'volume', 'turnover']

stocks = []

for stock in tqdm(df['tic'].unique()):
    tmp = df[df['tic'] == stock].reset_index(drop=True)
    tmp[cols_to_normalize] = scaler.fit_transform(tmp[cols_to_normalize])
    tmp = tmp.drop(columns=['date', 'tic'])
    # tmp = tmp.drop(columns=['date', 'tic', 'sw_kdcode_1'])
    stocks.append(tmp.values)

stocks = np.array(stocks)

100%|██████████| 116/116 [00:01<00:00, 94.84it/s]


In [68]:
df['ret_1'] = df['close'] / df['close'].shift(1)
df['ret_1'] = df['ret_1'].fillna(1)
df.loc[df['date'] == '2018-01-02', 'ret_1'] = 1
df['ret_1'] = df['ret_1'] - 1

In [69]:
ret = []

for stock in tqdm(df['tic'].unique()):
    tmp = df[df['tic'] == stock].reset_index(drop=True)
    ret.append(tmp['ret_1'].values)

ret = np.array(ret)

100%|██████████| 116/116 [00:00<00:00, 121.84it/s]


In [70]:
stocks.shape

(116, 1457, 6)

In [71]:
ret.shape

(116, 1457)

In [73]:
stocks[0]

array([[0.14572666, 0.14574257, 0.14129159, 0.15429282, 0.0210878 ,
        0.0374774 ],
       [0.14966522, 0.14772277, 0.14559687, 0.15719618, 0.02642707,
        0.04722418],
       [0.15833005, 0.15564356, 0.15577299, 0.16590626, 0.05963536,
        0.10817332],
       ...,
       [0.31547853, 0.31683168, 0.31076321, 0.33181253, 0.01440102,
        0.01091871],
       [0.32099252, 0.31762376, 0.31741683, 0.33471589, 0.02350612,
        0.02174417],
       [0.31981095, 0.32435644, 0.31819961, 0.3405226 , 0.02339496,
        0.02166309]])

In [75]:
np.save(f'{code}/stocks_data.npy', stocks)

In [74]:
np.save(f'{code}/ror.npy', ret)

In [43]:
dict_industry = {
            '110000': '农林牧渔',
            '220000': '基础化工',
            '230000': '钢铁',   
            '240000': '有色金属',  
            '270000': '电子',  
            '280000': '汽车',   
            '330000': '家用电器',  
            '340000': '食品饮料',   
            '350000': '纺织服饰',   
            '360000': '轻工制造',   
            '370000': '医药生物',   
            '410000': '公用事业',   
            '420000': '交通运输',   
            '430000': '房地产',  
            '450000': '商贸零售',  
            '460000': '社会服务',   
            '480000': '银行',   
            '490000': '非银金融',  
            '510000': '综合',  
            '610000': '建筑材料',  
            '620000': '建筑装饰',  
            '630000': '电力设备',  
            '640000': '机械设备',  
            '650000': '国防军工',
            '710000': '计算机',  
            '720000': '传媒',  
            '730000': '通信',  
            '740000': '煤炭', 
            '210000': '煤炭',
            '750000': '石油石化',  
            '760000': '环保',  
            '770000': '美容护理',  
        }

dict = dict_industry

In [44]:
matrix = np.zeros((stock_num, stock_num))
for i, stock1 in enumerate(stock_unique):
    for j, stock2 in enumerate(stock_unique):
        if dict.get(str(unique_mapping.get(stock1))) == dict.get(str(unique_mapping.get(stock2))):
            matrix[i][j] = 1
np.fill_diagonal(matrix, 1)

In [45]:
matrix.shape

(140, 140)

In [46]:
np.count_nonzero(matrix)

1298

In [47]:
np.save(f'{code}/industry_classification.npy', matrix)